In [1]:
import collections
import itertools
import json
import numpy as np
import os
import pathlib
import subprocess
import sys
import transformers
from typing import *

import markdown_strings
from IPython.display import display, Markdown, Latex

try:
    from rich import pretty
    pretty.install()
    from rich import print
except ImportError:
    pass

import tensorflow as tf
import tqdm.notebook as tqdm

_PROJECT_DIRECTORY = pathlib.Path().resolve().parent
sys.path.append(str(_PROJECT_DIRECTORY))
import constants
import utils

In [2]:
def normal(text, escape=False):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(text))

def h1(text, escape=False):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(f"# {text}"))
    
def h2(text, escape=False):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(f"#### {text}"))
    
def quote(text, escape=True):
    if escape:
        text = markdown_strings.esc_format(text)
    display(Markdown(markdown_strings.blockquote(text)))

In [3]:
def build_per_split(num_paths_display):
    h1("Getting filenames.")
    h2("Loading json config.")
    config_path = _PROJECT_DIRECTORY/"configs"/"train_configs"/"tpu_gpt2_eli5_kilt.json"
    config = utils.from_json_file(config_path)
    h2("Calling `gsutil ls` on the dataset repo.")
    ds_path = config["tfr_prefix"]
    filenames = subprocess.check_output(f"gsutil ls {ds_path}", shell=True).decode().strip().split("\n")
    h2("A few paths:")
    normal(f"There are actually {len(filenames)}.")
    normal(" - " + "\n - ".join(filenames[:num_paths_display]))
    

    h1("Building the `per_split` Path dict.")
    per_split = collections.defaultdict(list)
    for path in tqdm.tqdm(filenames, desc="Building `per_split` dict."):
        split = pathlib.Path(path).name.split("_")[0]
        per_split[split].append(path)

    normal("Sorting the `per_split` lists.")
    for split in per_split:
        # Ad-hoc split per file index
        per_split[split].sort(key=lambda p: int(pathlib.Path(p).name.split("_")[1].split(".")[0]))

    normal("Len per split for the per_split dict:")
    print({split: len(per_split[split]) for split in per_split})
    return per_split

In [4]:
def build_dataset(paths, context_window_size, split):
    ds = tf.data.TFRecordDataset(paths)


    description = {
      constants.CTH5Fields.distances:
          tf.io.FixedLenFeature((), tf.string),
      constants.CTH5Fields.gpt2_retrieved_ids:
          tf.io.FixedLenFeature((), tf.string),
      constants.CTH5Fields.gpt2_question_ids_inputs:
          tf.io.FixedLenFeature((), tf.string),
    }
    if split != constants.SplitChoices.test:
        description[
            constants.CTH5Fields.gpt2_answer_ids_inputs
        ] = tf.io.FixedLenFeature((), tf.string)

    feature_dtypes = {
      constants.CTH5Fields.distances:
          tf.float32,
      constants.CTH5Fields.gpt2_retrieved_ids:
          tf.int32,
      constants.CTH5Fields.gpt2_question_ids_inputs:
          tf.int32,
    }
    if split != constants.SplitChoices.test:
        feature_dtypes[
            constants.CTH5Fields.gpt2_answer_ids_inputs
        ] = tf.int32

    feature_shape = {
      constants.CTH5Fields.distances:
          (10,),
      constants.CTH5Fields.gpt2_retrieved_ids:
          (10, context_window_size,),
      constants.CTH5Fields.gpt2_question_ids_inputs:
          (context_window_size,),
    }
    if split != constants.SplitChoices.test:
        feature_shape[constants.CTH5Fields.gpt2_answer_ids_inputs] = (
            context_window_size
        )

    @tf.function
    def parse(sample):
        example = tf.io.parse_single_example(sample, description)
        output = {}
        for k, v in example.items():
            output[k] = tf.io.parse_tensor(v, out_type=feature_dtypes[k])
            output[k].set_shape(feature_shape[k])
        return output

    ds = ds.map(
      parse,
      num_parallel_calls=tf.data.experimental.AUTOTUNE,
      deterministic=False
      )
    return ds


In [ ]:
def decode_line(tokenizer, line):
    return tokenizer.decode([x for x in line if x >= 0])

def count(paths, context_window_size, split, tokenizer, min_length=7):
    ds = build_dataset(paths, context_window_size, split)
    i = 0    
    feature_lengths = collections.defaultdict(lambda: collections.defaultdict(int))
    
    for item in tqdm.tqdm(ds, desc=f"Counting items for split `{split}`"):
        i += 1
        for feature_key, feature in item.items():
            
            if feature_key in {
                constants.CTH5Fields.gpt2_question_ids_inputs, constants.CTH5Fields.gpt2_answer_ids_inputs
            }:
                length = np.sum(feature >= 0)
                feature_lengths[feature_key][length] += 1
#                 if length <= min_length:
#                     normal(f"{split} - `{feature_key}`: {decode_line(tokenizer, feature)}")

    for k, v in feature_lengths.items():
        print(f"{split} - {k}: ")
        sorted_ = sorted(v.items(), key=lambda x: x[0])
        print(sorted_)
    return i


def main():
    _MAX_QTY = None
    _MODEL_TYPE = "gpt2-xl"
    _MODEL_CONFIG = transformers.AutoConfig.from_pretrained("distilgpt2")
    _CONTEXT_WINDOW_SIZE = _MODEL_CONFIG.n_ctx
    _EXPECTED_SIZES = dict(train=272634, eval=1507, test=600)

    tokenizer = transformers.AutoTokenizer.from_pretrained(_MODEL_TYPE)
    per_split = build_per_split()

    for split in ["eval", "test", "train"]:
        to_test = per_split[split][:_MAX_QTY]
        paths = tf.data.Dataset.from_tensor_slices(to_test)
        count(paths, _CONTEXT_WINDOW_SIZE, split, tokenizer)
main()

# Getting filenames.

#### Loading json config.

#### Calling `gsutil ls` on the dataset repo.

#### A few paths:

 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_0.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_10.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_100.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1000.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1001.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1002.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1003.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1004.tfr
 - gs://julesgm-research-v3/tfrecord_query_cache/20210225-191356/eval_1005.tfr

...

{ 'eval' : 2048 , 'test' : 2048 , 'train' : 2048 , 'validation' : 2048 }

gpt2_answer_ids:

[ 
 ( 9 , 5 ) ,
 ( 10 , 1 ) ,
 ( 11 , 3 ) ,
 ( 12 , 1 ) ,
 ( 13 , 3 ) ,
 ( 14 , 2 ) ,
 ( 15 , 1 ) ,
 ( 16 , 8 ) ,
 ( 17 , 3 ) ,
 ( 18 , 3 ) ,
 ( 19 , 5 ) ,
 ( 20 , 4 ) ,
 ( 21 , 6 ) ,
 ( 22 , 2 ) ,
 ( 23 , 13 ) ,
 ( 24 , 5 ) ,
 ( 25 , 10 ) ,
 ( 26 , 12 ) ,
 ( 27 , 6 ) ,
 ( 28 , 4 ) ,
 ( 29 , 16 ) ,
 ( 30 , 5 ) ,
 ( 31 , 14 ) ,
 ( 32 , 10 ) ,
 ( 33 , 10 ) ,
 ( 34 , 14 ) ,
 ( 35 , 10 ) ,
 ( 36 , 7 ) ,
 ( 37 , 13 ) ,
 ( 38 , 9 ) ,
 ( 39 , 10 ) ,
 ( 40 , 5 ) ,
 ( 41 , 11 ) ,
 ( 42 , 12 ) ,
 ( 43 , 4 ) ,
 ( 44 , 10 ) ,
 ( 45 , 5 ) ,
 ( 46 , 10 ) ,
 ( 47 , 8 ) ,
 ( 48 , 10 ) ,
 ( 49 , 13 ) ,
 ( 50 , 9 ) ,
 ( 51 , 11 ) ,
 ( 52 , 6 ) ,
 ( 53 , 6 ) ,
 ( 54 , 14 ) ,
 ( 55 , 8 ) ,
 ( 56 , 3 ) ,
 ( 57 , 8 ) ,
 ( 58 , 12 ) ,
 ( 59 , 6 ) ,
 ( 60 , 8 ) ,
 ( 61 , 6 ) ,
 ( 62 , 7 ) ,
 ( 63 , 6 ) ,
 ( 64 , 12 ) ,
 ( 65 , 9 ) ,
 ( 66 , 6 ) ,
 ( 67 , 11 ) ,
 ( 68 , 11 ) ,
 ( 69 , 5 ) ,
 ( 70 , 7 ) ,
 ( 71 , 9 ) ,
 ( 72 , 9 ) ,
 ( 73 , 8 ) ,
 ( 74 , 7 ) ,
 ( 75 , 5 ) ,
 ( 76 , 4 ) ,
 ( 77 , 8 ) ,
 ( 78 , 13 ) ,
 ( 79 , 11 ) ,
 ( 80 , 10 ) ,
 ( 81 , 10 ) ,
 ( 82 , 12 ) ,
 ( 83 , 12 ) ,
 ( 84 , 9 ) ,
 ( 85 , 10 ) ,
 ( 86 , 4 ) ,
 ( 87 , 6 ) ,
 ( 88 , 6 ) ,
 ( 89 , 10 ) ,
 ( 90 , 6 ) ,
 ( 91 , 6 ) ,
 ( 92 , 10 ) ,
 ( 93 , 5 ) ,
 ( 94 , 6 ) ,
 ( 95 , 7 ) ,
 ( 96 , 12 ) ,
 ( 97 , 6 ) ,
 ( 98 , 10 ) ,
 ( 99 , 5 ) ,
 ( 100 , 8 ) ,
 ( 101 , 4 ) ,
 ( 102 , 9 ) ,
 ( 103 , 10 ) ,
 ( 104 , 7 ) ,
 ( 105 , 9 ) ,
 ( 106 , 6 ) ,
 ( 107 , 6 ) ,
 ( 108 , 4 ) ,
 ( 109 , 3 ) ,
 ( 110 , 6 ) ,
 ( 111 , 4 ) ,
 ( 112 , 5 ) ,
 ( 113 , 7 ) ,
 ( 114 , 10 ) ,
 ( 115 , 6 ) ,
 ( 116 , 4 ) ,
 ( 117 , 9 ) ,
 ( 118 , 6 ) ,
 ( 119 , 5 ) ,
 ( 120 , 6 ) ,
 ( 121 , 8 ) ,
 ( 122 , 9 ) ,
 ( 123 , 2 ) ,
 ( 124 , 3 ) ,
 ( 125 , 5 ) ,
 ( 126 , 7 ) ,
 ( 127 , 10 ) ,
 ( 128 , 5 ) ,
 ( 129 , 7 ) ,
 ( 130 , 2 ) ,
 ( 131 , 8 ) ,
 ( 132 , 2 ) ,
 ( 133 , 6 ) ,
 ( 134 , 5 ) ,
 ( 135 , 4 ) ,
 ( 136 , 4 ) ,
 ( 137 , 5 ) ,
 ( 138 , 5 ) ,
 ( 139 , 10 ) ,
 ( 140 , 5 ) ,
 ( 141 , 3 ) ,
 ( 142 , 2 ) ,
 ( 143 , 4 ) ,
 ( 144 , 4 ) ,
 ( 145 , 3 ) ,
 ( 146 , 7 ) ,
 ( 147 , 5 ) ,
 ( 148 , 3 ) ,
 ( 149 , 4 ) ,
 ( 150 , 4 ) ,
 ( 152 , 6 ) ,
 ( 153 , 10 ) ,
 ( 154 , 1 ) ,
 ( 155 , 4 ) ,
 ( 156 , 6 ) ,
 ( 157 , 1 ) ,
 ( 158 , 6 ) ,
 ( 159 , 8 ) ,
 ( 160 , 4 ) ,
 ( 161 , 5 ) ,
 ( 162 , 2 ) ,
 ( 163 , 5 ) ,
 ( 164 , 1 ) ,
 ( 165 , 6 ) ,
 ( 166 , 7 ) ,
 ( 167 , 5 ) ,
 ( 168 , 2 ) ,
 ( 169 , 4 ) ,
 ( 170 , 5 ) ,
 ( 171 , 2 ) ,
 ( 172 , 2 ) ,
 ( 173 , 5 ) ,
 ( 175 , 3 ) ,
 ( 176 , 1 ) ,
 ( 177 , 1 ) ,
 ( 178 , 3 ) ,
 ( 179 , 7 ) ,
 ( 181 , 3 ) ,
 ( 182 , 2 ) ,
 ( 183 , 2 ) ,
 ( 184 , 3 ) ,
 ( 185 , 1 ) ,
 ( 186 , 1 ) ,
 ( 187 , 2 ) ,
 ( 188 , 2 ) ,
 ( 190 , 4 ) ,
 ( 191 , 2 ) ,
 ( 193 , 4 ) ,
 ( 194 , 1 ) ,
 ( 195 , 2 ) ,
 ( 196 , 3 ) ,
 ( 197 , 2 ) ,
 ( 198 , 3 ) ,
 ( 199 , 3 ) ,
 ( 200 , 3 ) ,
 ( 201 , 4 ) ,
 ( 202 , 1 ) ,
 ( 203 , 5 ) ,
 ( 204 , 4 ) ,
 ( 206 , 6 ) ,
 ( 208 , 4 ) ,
 ( 210 , 1 ) ,
 ( 211 , 1 ) ,
 ( 213 , 2 ) ,
 ( 214 , 2 ) ,
 ( 216 , 3 ) ,
 ( 217 , 4 ) ,
 ( 219 , 1 ) ,
 ( 220 , 2 ) ,
 ( 221 , 2 ) ,
 ( 222 , 1 ) ,
 ( 223 , 2 ) ,
 ( 224 , 1 ) ,
 ( 225 , 2 ) ,
 ( 226 , 1 ) ,
 ( 227 , 2 ) ,
 ( 228 , 3 ) ,
 ( 229 , 1 ) ,
 ( 231 , 3 ) ,
 ( 234 , 1 ) ,
 ( 235 , 1 ) ,
 ( 237 , 3 ) ,
 ( 238 , 1 ) ,
 ( 240 , 2 ) ,
 ( 241 , 1 ) ,
 ( 242 , 1 ) ,
 ( 243 , 1 ) ,
 ( 244 , 2 ) ,
 ( 246 , 1 ) ,
 ( 247 , 4 ) ,
 ( 249 , 2 ) ,
 ( 252 , 1 ) ,
 ( 254 , 3 ) ,
 ( 255 , 2 ) ,
 ( 256 , 4 ) ,
 ( 257 , 1 ) ,
 ( 260 , 2 ) ,
 ( 261 , 1 ) ,
 ( 262 , 3 ) ,
 ( 263 , 1 ) ,
 ( 264 , 3 ) ,
 ( 266 , 3 ) ,
 ( 268 , 1 ) ,
 ( 269 , 1 ) ,
 ( 270 , 3 ) ,
 ( 271 , 2 ) ,
 ( 272 , 2 ) ,
 ( 273 , 4 ) ,
 ( 274 , 1 ) ,
 ( 275 , 2 ) ,
 ( 276 , 2 ) ,
 ( 278 , 1 ) ,
 ( 279 , 1 ) ,
 ( 280 , 2 ) ,
 ( 281 , 1 ) ,
 ( 283 , 3 ) ,
 ( 285 , 1 ) ,
 ( 286 , 1 ) ,
 ( 287 , 2 ) ,
 ( 288 , 2 ) ,
 ( 289 , 1 ) ,
 ( 290 , 2 ) ,
 ( 291 , 1 ) ,
 ( 292 , 1 ) ,
 ( 294 , 1 ) ,
 ( 295 , 2 ) ,
 ( 296 , 1 ) ,
 ( 297 , 3 ) ,
 ( 298 , 1 ) ,
 ( 300 , 3 ) ,
 ( 301 , 2 ) ,
 ( 304 , 1 ) ,
 ( 305 , 2 ) ,
 ( 306 , 2 ) ,
 ( 307 , 1 ) ,
 ( 308 , 1 ) ,
 ( 309 , 2 ) ,
 ( 310 , 1 ) ,


gpt2_question_ids:

[ 
 ( 2 , 1 ) ,
 ( 3 , 5 ) ,
 ( 4 , 10 ) ,
 ( 5 , 17 ) ,
 ( 6 , 24 ) ,
 ( 7 , 26 ) ,
 ( 8 , 48 ) ,
 ( 9 , 54 ) ,
 ( 10 , 46 ) ,
 ( 11 , 61 ) ,
 ( 12 , 59 ) ,
 ( 13 , 76 ) ,
 ( 14 , 64 ) ,
 ( 15 , 60 ) ,
 ( 16 , 69 ) ,
 ( 17 , 63 ) ,
 ( 18 , 56 ) ,
 ( 19 , 64 ) ,
 ( 20 , 49 ) ,
 ( 21 , 53 ) ,
 ( 22 , 70 ) ,
 ( 23 , 39 ) ,
 ( 24 , 44 ) ,
 ( 25 , 44 ) ,
 ( 26 , 43 ) ,
 ( 27 , 52 ) ,
 ( 28 , 33 ) ,
 ( 29 , 21 ) ,
 ( 30 , 19 ) ,
 ( 31 , 25 ) ,
 ( 32 , 18 ) ,
 ( 33 , 18 ) ,
 ( 34 , 26 ) ,
 ( 35 , 21 ) ,
 ( 36 , 10 ) ,
 ( 37 , 14 ) ,
 ( 38 , 13 ) ,
 ( 39 , 13 ) ,
 ( 40 , 7 ) ,
 ( 41 , 15 ) ,
 ( 42 , 6 ) ,
 ( 43 , 2 ) ,
 ( 44 , 3 ) ,
 ( 45 , 6 ) ,
 ( 46 , 7 ) ,
 ( 47 , 3 ) ,
 ( 48 , 5 ) ,
 ( 49 , 1 ) ,
 ( 50 , 3 ) ,
 ( 51 , 2 ) ,
 ( 52 , 1 ) ,
 ( 54 , 4 ) ,
 ( 55 , 4 ) ,
 ( 58 , 1 ) ,
 ( 59 , 2 ) ,
 ( 60 , 1 ) ,
 ( 61 , 1 ) ,
 ( 62 , 2 ) ,
 ( 64 , 1 ) ,
 ( 65 , 1 ) ,
 ( 68 , 1 ) 
 ]

In [ ]:
def display_item(item):    
    h2("Question:")
    quote(decode_line(item[constants.CTH5Fields.gpt2_question_ids_inputs]))

    if _SPLIT != "test":
        h2("Answer:")
        quote(decode_line(item[constants.CTH5Fields.gpt2_answer_ids_inputs]))

    h2("Retrieved segments:")
    for line in item[constants.CTH5Fields.gpt2_retrieved_ids]:
        quote(decode_line(line))

